<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `DFS` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [1]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from Greedy import Greedy
from pyrat import BigHolesRandomMaze, Action, Graph, Game

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [2]:
import unittest
import random
from typing import Dict, List, Any

class GreedyTests(unittest.TestCase):
    """
    This class tests the methods of the Greedy class.
    For each method, we test it with a few different configurations.
    """

    def test_ajout_sommets2(self: Self) -> None:
        """
        This method tests the 'ajout_sommets' method.
        Checks:
        - The graph is updated correctly.
        - All cheeses and the initial location are present in the graph.
        - The graph structure is consistent (no unexpected edges or data).
        """
        # Constants
        TESTS = [
            {"initial_location": 0, "cheeses": [1, 2, 3, 4, 5]},
            {"initial_location": 1, "cheeses": [0, 2, 3, 4, 5]},
            {"initial_location": 2, "cheeses": [0, 1, 3, 4, 5]},
            {"initial_location": 3, "cheeses": [0, 1, 2, 4, 5]},
            {"initial_location": 4, "cheeses": [0, 1, 2, 3, 5]},
            {"initial_location": 5, "cheeses": [0, 1, 2, 3, 4]},
        ]

        # Test on several controlled examples
        for i, test_case in enumerate(TESTS):
            # Instantiate the player
            player = Greedy()

            # Get the input data
            initial_location = test_case["initial_location"]
            cheeses = test_case["cheeses"]

            # Create the graph
            graph = {}
            player.ajout_sommet2(graph, initial_location, cheeses)

            # Assertions
            # Check if all cheeses and the initial location are in the graph
            for cheese in cheeses:
                self.assertIn(cheese, graph, f"Cheese {cheese} missing in graph for test {i}.")
            self.assertIn(initial_location, graph, f"Initial location {initial_location} missing in graph for test {i}.")

            # Check if all graph nodes have empty dictionaries
            for node, edges in graph.items():
                self.assertEqual(edges, {}, f"Unexpected edges found for node {node} in test {i}.")


    def setUp(self):
        """
        Initialisation d'un joueur Greedy pour les tests.
        """
        self.player = Greedy("Player1")  # Crée un joueur Greedy
        self.initial_location = (0, 0)  # Exemple de position initiale
        self.cheeses = [(1, 1), (2, 2)]  # Exemple de positions de fromages
    def test_find_route(self):
        """
        Teste la méthode find_route pour vérifier que le chemin est correctement généré.
        """
        # Préparer l'environnement de test
        player = Greedy()  # L'instance de la classe Greedy (ou autre)
        
        # Exemples de routing_table avec un plus grand nombre de noeuds
        routing_table = {
            97: {97: None, 112: 97, 82: 97, 98: 97, 127: 112, 113: 114, 81: 82, 83: 82, 67: 68, 99: 98},
            112: {112: None, 113: 112, 115: 114},  # Exemple supplémentaire
            82: {82: None, 83: 82, 97: 98},
            127: {127: None, 113: 114},
        }

        source = 97
        path = [97, 112, 127]  # Chemin à vérifier (97 -> 112 -> 127)

        # Appel à la méthode find_route
        result = player.find_route(routing_table, source, path)

        # Le chemin attendu : de 97 à 112, puis de 112 à 127
        expected_route = [97, 112, 127]

        # Vérification de l'égalité entre le résultat et le chemin attendu
        self.assertEqual(result, expected_route)

    def test_greedy_with_unreachable_cheese(self):
        """
        Tester que la fonction greedy échoue si certains fromages ne peuvent pas être atteints.
        """
        # Création d'un graphe où un fromage est inaccessibile
        graph = {
            self.initial_location: {self.cheeses[0]: 2},
            self.cheeses[0]: {},
            self.cheeses[1]: {}  # Pas de chemin vers le deuxième fromage
        }
        
        # Appel à la fonction greedy avec une exception attendue
        with self.assertRaises(ValueError):
            self.player.greedy(self.cheeses, self.initial_location, graph)
            
    def test_greedy_empty_graph(self):
        """
        Tester que la fonction greedy échoue avec un graphe vide ou un graphe sans fromage.
        """
        # Création d'un graphe vide
        graph = {self.initial_location: {}}
        
        # Appel à la fonction greedy avec une exception attendue
        with self.assertRaises(ValueError):
            self.player.greedy(self.cheeses, self.initial_location, graph)

    def test_greedy_single_cheese(self):
        """
        Tester que la fonction greedy fonctionne correctement avec un seul fromage.
        """
        # Création d'un graphe avec un seul fromage
        graph = {
            self.initial_location: {self.cheeses[0]: 3},
            self.cheeses[0]: {}
        }
        
        # Appel à la fonction greedy
        visited, distance_totale = self.player.greedy([self.cheeses[0]], self.initial_location, graph)
        
        # Vérifier que le fromage est visité
        self.assertEqual(len(visited), 2)  # +1 pour la position initiale
        # Vérifier la distance totale (3)
        self.assertEqual(distance_totale, 3)



    def test_greedy_with_disconnected_graph(self):
        """
        Tester que la fonction greedy échoue si le graphe est non connexe et certains fromages sont inaccessibles.
        """
        # Création d'un graphe non connexe (le fromage (2, 2) est inaccessibile)
        graph = {
            self.initial_location: {(1, 1): 2},
            (1, 1): {},
            (2, 2): {}  # Le fromage (2, 2) est inaccessibile
        }
        
        # Appel à la fonction greedy avec une exception attendue
        with self.assertRaises(ValueError):
            self.player.greedy([(1, 1), (2, 2)], self.initial_location, graph)

    def test_greedy_cheeses_in_chain(self):
        """
        Tester que la fonction greedy visite les fromages dans une disposition en chaîne et calcule correctement la distance totale.
        """
        # Création d'un graphe où les fromages sont connectés en chaîne
        graph = {
            self.initial_location: {(1, 1): 2},
            (1, 1): {(2, 2): 3},
            (2, 2): {(3, 3): 1},
            (3, 3): {}
        }
        
        # Appel à la fonction greedy
        visited, distance_totale = self.player.greedy([(1, 1), (2, 2), (3, 3)], self.initial_location, graph)
        
        # Vérifier que tous les fromages sont visités
        self.assertEqual(len(visited), 4)  # +1 pour la position initiale
        # Vérifier la distance totale (2 + 3 + 1 = 6)
        self.assertEqual(distance_totale, 6)

    def test_greedy_2_cheeses(self):
        """
        Tester que la fonction greedy fonctionne correctement avec plusieurs fromages.
        """
        # Création d'un graphe avec 2 fromages
        graph = {
            self.initial_location: {self.cheeses[0]: 2, self.cheeses[1]: 3},
            self.cheeses[0]: {self.cheeses[1]: 1},
            self.cheeses[1]: {}
        }
        
        # Appel à la fonction greedy
        visited, distance_totale = self.player.greedy(self.cheeses, self.initial_location, graph)
        
        # Vérifier que tous les fromages sont visités
        self.assertEqual(len(visited)-1, 2)

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [3]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False) 

test_ajout_sommets2 (__main__.GreedyTests.test_ajout_sommets2)
This method tests the 'ajout_sommets' method. ... ok
test_find_route (__main__.GreedyTests.test_find_route)
Teste la méthode find_route pour vérifier que le chemin est correctement généré. ... ERROR
test_greedy_2_cheeses (__main__.GreedyTests.test_greedy_2_cheeses)
Tester que la fonction greedy fonctionne correctement avec plusieurs fromages. ... ERROR
test_greedy_cheeses_in_chain (__main__.GreedyTests.test_greedy_cheeses_in_chain)
Tester que la fonction greedy visite les fromages dans une disposition en chaîne et calcule correctement la distance totale. ... ERROR
test_greedy_empty_graph (__main__.GreedyTests.test_greedy_empty_graph)
Tester que la fonction greedy échoue avec un graphe vide ou un graphe sans fromage. ... ERROR
test_greedy_single_cheese (__main__.GreedyTests.test_greedy_single_cheese)
Tester que la fonction greedy fonctionne correctement avec un seul fromage. ... ERROR
test_greedy_with_disconnected_graph (__m

Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
